# Experiment
Fine tuning NVIDIA STT Fast Conformer Transducer

This time with a hopefully better audio processing.

In [35]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch
    - pytorch-cuda=11.8
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cuda-cudart-11.8.89        |                0         197 KB  nvidia
    cuda-cupti-11.8.87         |                0        25.3 MB  nvidia
    cuda-libraries-11.8.0      |                0           1 KB  nvidia
    cuda-nvrtc-11.8.89         |                0        19.1 MB  nvidia
    cuda-nvtx-11.8.86          |                0          57 KB  nvidia
    cuda-runtime-11.8.0        |                0           1 KB  nvidia
    libcublas-11.11.3.6        |          

In [42]:
!conda install cudatoolkit -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudatoolkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.8.0         |       h6a678d5_0       630.7 MB
    ------------------------------------------------------------
                                           Total:       630.7 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        pkgs/main/linux-64::cudatoolkit-11.8.0-h6a678d5_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2023.5.7~ --> pkgs/main::ca-certificates-2023.01.10-h06a4308_0 
  certifi            conda-forge/noarch::certifi-2023.5.7-~ --> pkgs/main/linux-64::certifi-2023.5.7-py310h06a4308_0 
  conda              conda-forge::conda-23.3.1-py310hff520~ --> pkgs/main::conda-23.3.1-py310h06a4308_0 




In [ ]:
!apt install build-essential -y

In [3]:
%pip install Cython
%pip install nemo_toolkit['all']

  Using cached Cython-0.29.34-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached nemo_toolkit-1.18.1-py3-none-any.whl (2.3 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Using cached wget-3.2-py3-none-any.whl
  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached onnx-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.6 MB)
  Using cached setuptools-65.5.1-py3-none-any.whl (1.2 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached numba-0.57.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.6 MB)
  Using cached wrapt-1.15.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (78 kB)
  Using cached gdown-4.7.1-py3-none-an

In [4]:
%pip install jiwer librosa nemo_text_processing noisereduce

Note: you may need to restart the kernel to use updated packages.


In [9]:
!conda install -y -c conda-forge libstdcxx-ng=12

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - libstdcxx-ng=12


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       hbcca054_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    conda-23.3.1               |  py310hff52083_0         941 KB  conda-forge
    libstdcxx-ng-12.2.0        |      h46fd767_19         4.3 MB  conda-forge
    python_abi-3.10            |          2_cp310           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.10-2_cp310 

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificate

In [56]:
%pip install pyloudnorm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 15.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=94938acb28f18591529e63bf2831a21d3194814df7273373ebe670dc204e2ae5
  Stored in directory: /root/.cache/pip/wheels/5e/a9/47/f118e66afd12240e4662752cc22cefae5d97275623aa8ef57d
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import copy
import json
import math
import librosa
import numpy as np
import pandas as pd

# Model Training
import nemo.collections.asr as nemo_asr
# NeMo's Experiment manager to handle checkpoint saving and logging
from nemo.utils import exp_manager
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer, seed_everything

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

# Model Evaluation
from jiwer import wer

# Reproducibility
seed_everything(42, workers=True)

[NeMo W 2023-05-23 04:49:20 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-23 04:49:21 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Global seed set to 42


42

### Load Dataset

In [2]:
# Define paths to data sources
DATASET_TRAIN = 'data/train/Train.csv'
FILE_TEST = 'data/test/SampleSubmission_Advanced.csv'
TRAIN_DATA_DIR = 'data/train/'
TEST_DATA_DIR = 'data/test/'

In [3]:
# Load dataset
dataset = pd.read_csv(DATASET_TRAIN)
# Shuffle the records
dataset = dataset.sample(frac=1, random_state=1).reset_index(drop=True)
# Load file for storing model predictions
test_set = pd.read_csv(FILE_TEST)

In [4]:
dataset["annotation"] = dataset["annotation"].str.lower() # normalize to lower case

In [5]:
# Define train test split
TRAIN_TEST_SPLIT = 0.8

# Split the dataset
train_proportion = int(TRAIN_TEST_SPLIT * len(dataset))
train_set = dataset[:train_proportion]
dev_set = dataset[train_proportion:]

In [6]:
# Build manifest files for training with NVIDIA STT models
def build_manifest(ds, manifest_path, is_train_set):
    with open(manifest_path, 'w') as mp:
        for idx, record in ds.iterrows():
            # Get audio file
            audio_path = record['path']
            if is_train_set:
                # Get duration
                duration = librosa.core.get_duration(filename=f'{TRAIN_DATA_DIR}{audio_path}')
                # Get transcript
                transcript = record['annotation'].lower() # normalize to lower case
                metadata = {
                    "audio_filepath": f'{TRAIN_DATA_DIR}{audio_path}',
                    "duration": duration,
                    "text": transcript
                }
            else:
                # Get duration 
                duration = librosa.core.get_duration(filename=f'{TEST_DATA_DIR}audio/{audio_path}')
                metadata = {
                    "audio_filepath": f'{TEST_DATA_DIR}audio/{audio_path}',
                    "duration": duration
                } 

            json.dump(metadata, mp)
            mp.write('\n')

# Create manifests file if they haven't been created
train_manifest_file = os.path.join(TRAIN_DATA_DIR, 'train_manifest.json')
if not os.path.isfile(train_manifest_file):
    build_manifest(train_set, train_manifest_file, is_train_set=True)

val_manifest_file = os.path.join(TRAIN_DATA_DIR, 'val_manifest.json')
if not os.path.isfile(val_manifest_file):
    build_manifest(dev_set, val_manifest_file, is_train_set=True)

test_manifest_file = os.path.join(TEST_DATA_DIR, 'test_manifest.json')
if not os.path.isfile(test_manifest_file):
    build_manifest(test_set, test_manifest_file, is_train_set=False)

print('***Manifests created***')

***Manifests created***


### Prepare Model

#### Tokenizer

In [7]:
# Create directory for nemo scripts
if not os.path.exists("scripts/"):
  os.makedirs("scripts")

In [8]:
# Leverage modern encoder architecture like Conformer-T through a tokenizer based model
if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py

In [11]:
VOCAB_SIZE = 32000
TOKENIZER_TYPE = "spe"

if not os.path.exists("tokenizers"):
  os.makedirs("tokenizers")

!python scripts/process_asr_text_tokenizer.py \
   --manifest=$train_manifest_file \
   --data_root="tokenizers" \
   --tokenizer=$TOKENIZER_TYPE \
   --log \
   --vocab_size=$VOCAB_SIZE

[NeMo W 2023-05-23 04:54:37 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
usage: process_asr_text_tokenizer.py [-h]
                                     (--manifest MANIFEST | --data_file DATA_FILE)
                                     --data_root DATA_ROOT
                                     [--vocab_size VOCAB_SIZE]
                                     [--tokenizer {spe,wpe}]
                                     [--spe_type {bpe,unigram,char,word}]
                                     [--spe_character_coverage SPE_CHARACTER_COVERAGE]
                                     [--spe_bos] [--spe_eos] [--spe_pad]
                                     [--spe_sample_size SPE_SAMPLE_SIZE]
                                     [--spe_train_extremely_large_corpus]
                                     [--spe_max_sentencepiece_length SPE_MAX_SENTENCEPIECE_LENGTH]
                                     [--spe_no_split_by_unicode_script]
                      

In [13]:
# Tokenizer path
VOCAB_SIZE = 32000
TOKENIZER_TYPE = "bpe"

TOKENIZER = os.path.join("tokenizers", f"tokenizer_spe_bpe_v32000")
TOKENIZER_TYPE_CFG = "bpe"

#### Pretrained Model

In [74]:
checkpoint = "experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision.nemo"
# if checkpoint is None:
#     asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_en_fastconformer_transducer_large')

load_asr_model = nemo_asr.models.ASRModel.restore_from(checkpoint)

[NeMo I 2023-05-23 07:10:47 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-23 07:10:47 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: data/train/train_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: true
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    normalize_transcripts: false
    
[NeMo W 2023-05-23 07:10:47 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: data/train/val_manifest.json
    sa

[NeMo I 2023-05-23 07:10:47 features:287] PADDING: 0


[NeMo W 2023-05-23 07:10:48 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-05-23 07:10:49 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-23 07:10:50 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /home/experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision.nemo.


#### Freeze Encoder
Since the TIL dataset is quite small, it is better to freeze the encoder.

In [15]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

In [16]:
freeze_encoder = True
if freeze_encoder:
    asr_model.encoder.freeze()
    asr_model.encoder.apply(enable_bn_se)
    print("Model encoder has been frozen")
else:
    asr_model.encoder.unfreeze()
    print("Model encoder has been un-frozen")

Model encoder has been frozen


### Configure Model

In [17]:
cfg = copy.deepcopy(asr_model.cfg)

In [18]:
cfg.tokenizer.dir = TOKENIZER
cfg.tokenizer.type = TOKENIZER_TYPE_CFG

# Set tokenizer config
asr_model.cfg.tokenizer = cfg.tokenizer

In [19]:
print(OmegaConf.to_yaml(cfg.tokenizer))

dir: tokenizers/tokenizer_spe_bpe_v32000
type: bpe
model_path: nemo:e7bbc43ca90943c989b60aece82c16c8_tokenizer.model
vocab_path: nemo:b7966c4186cd467b8e612a7d9384f273_vocab.txt
spe_tokenizer_vocab: nemo:0e7783691b714849aa760378530c1f39_tokenizer.vocab



In [20]:
# Setup train, validation, test configs
with open_dict(cfg):    
  # Train dataset
  cfg.train_ds.manifest_filepath = train_manifest_file
  cfg.train_ds.normalize_transcripts = False
  cfg.train_ds.batch_size = 32
  cfg.train_ds.num_workers = 8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.trim_silence = True

  # Validation dataset
  cfg.validation_ds.manifest_filepath = val_manifest_file
  cfg.validation_ds.normalize_transcripts = False
  cfg.validation_ds.batch_size = 8
  cfg.validation_ds.num_workers = 8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.trim_silence = True

  # Test dataset
  cfg.test_ds.manifest_filepath = test_manifest_file
  cfg.test_ds.normalize_transcripts = False
  cfg.test_ds.batch_size = 8
  cfg.test_ds.num_workers = 8
  cfg.test_ds.pin_memory = True
  cfg.test_ds.trim_silence = True

In [21]:
# Setup data loaders with new configs
asr_model.setup_training_data(cfg.train_ds)
asr_model.setup_validation_data(cfg.validation_ds)
asr_model.setup_test_data(cfg.test_ds)

[NeMo I 2023-05-23 04:55:20 collections:193] Dataset loaded with 3000 files totalling 3.31 hours
[NeMo I 2023-05-23 04:55:20 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-23 04:55:20 collections:193] Dataset loaded with 750 files totalling 0.82 hours
[NeMo I 2023-05-23 04:55:20 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-23 04:55:21 collections:193] Dataset loaded with 12000 files totalling 13.26 hours
[NeMo I 2023-05-23 04:55:21 collections:194] 0 files were filtered totalling 0.00 hours


In [22]:
with open_dict(asr_model.cfg.optim):
  asr_model.cfg.optim.lr = 5e-3 # from model config
  asr_model.cfg.optim.betas = [0.9, 0.98]  # from model config
  asr_model.cfg.optim.weight_decay = 1e-3  # Original weight decay
  asr_model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  asr_model.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
  asr_model.cfg.optim.sched.min_lr = 5e-4  # from model config

In [23]:
# Original optimizer + scheduler
print(OmegaConf.to_yaml(asr_model.cfg.optim))

name: adamw
lr: 0.005
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: null
  warmup_ratio: 0.05
  min_lr: 0.0005



In [24]:
asr_model.cfg.joint.fuse_loss_wer = True
asr_model.cfg.joint.fused_batch_size = 16

In [25]:
asr_model.cfg.compute_eval_loss = True

#### Configure Data Aug

In [26]:
augment_audio = False
if augment_audio:
    with open_dict(asr_model.cfg.spec_augment):
      asr_model.cfg.spec_augment.freq_masks = 2
      asr_model.cfg.spec_augment.freq_width = 25
      asr_model.cfg.spec_augment.time_masks = 2
      asr_model.cfg.spec_augment.time_width = 0.05
else:
    with open_dict(asr_model.cfg.spec_augment):
      asr_model.cfg.spec_augment.freq_masks = 0
      asr_model.cfg.spec_augment.time_masks = 0

In [24]:
print(OmegaConf.to_yaml(asr_model.cfg.spec_augment))

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
freq_masks: 0
time_masks: 0
freq_width: 27
time_width: 0.05



### Training

In [34]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

EPOCHS = 10

# Initialize a Trainer for the Transducer model
trainer = Trainer(devices=1,
                  precision=16, # mixed precision
                  accelerator=accelerator,
                  max_epochs=EPOCHS,
                  enable_checkpointing=False,
                  logger=False,
                  log_every_n_steps=5,
                  check_val_every_n_epoch=1,
                  deterministic=True)

# Setup model with the trainer
asr_model.set_trainer(trainer)

# Update model's internal configuration
asr_model.cfg = asr_model._cfg

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
!wandb login 23c74d0b24c250fc7fe6df6c59e5ee5bca3340cd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [31]:
%env WANDB_PROJECT=TIL2023-ASR
%env WANDB_NOTEBOOK_NAME=NVIDIA STT Fine Tune 22 May.ipynb

env: WANDB_PROJECT=TIL2023-ASR
env: WANDB_NOTEBOOK_NAME=NVIDIA STT Fine Tune 22 May.ipynb


In [35]:
# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"nvidia-stt-ft-02-spe-mixed-precision",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
    create_wandb_logger=True,
    wandb_logger_kwargs={
        "project" : "TIL2023-ASR",
        "name": "nvidia-stt-ft-02-spe-mixed-precision"
    }
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2023-05-23 04:57:10 exp_manager:374] Experiments will be logged at experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10
[NeMo I 2023-05-23 04:57:10 exp_manager:797] TensorboardLogger has been set up


[NeMo W 2023-05-23 04:57:10 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
      rank_zero_warn(
    


[NeMo I 2023-05-23 04:57:10 exp_manager:812] WandBLogger has been set up


In [36]:
%%time
trainer.fit(asr_model)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-05-23 04:57:18 modelPT:722] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.005
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2023-05-23 04:57:18 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f7e39d59090>" 
    will be used during training (effective maximum steps = 940) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.05
    min_lr: 0.0005
    max_steps: 940
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 108 M 
2 | decoder           | RNNTDecoder                       | 3.9 M 
3 | joint             | RNNTJoint                         | 1.4 M 
4 | loss              | RNNTLoss                          | 0     
5 | spec_augmentation | SpectrogramAugmentation           | 0     
6 | wer               | RNNTBPEWER                        | 0     
------------------------------------------------------------------------
5.4 M     Trainable params
108 M     Non-trainable params
114 M     Total params
228.191   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2023-05-23 04:57:29 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-23 04:57:29 rnnt_wer_bpe:395] reference :he received nine birthday cards in the mail
[NeMo I 2023-05-23 04:57:29 rnnt_wer_bpe:396] predicted :she received nine birthday cards in the mail
[NeMo I 2023-05-23 04:57:30 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-23 04:57:30 rnnt_wer_bpe:395] reference :she scored nine for the quiz
[NeMo I 2023-05-23 04:57:30 rnnt_wer_bpe:396] predicted :she scored nine for the queries


...

[NeMo I 2023-05-23 05:09:15 nemo_model_checkpoint:168] New best .nemo model saved to: /home/experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision.nemo
[NeMo I 2023-05-23 05:09:18 nemo_model_checkpoint:168] New best .nemo model saved to: /home/experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision.nemo


`Trainer.fit` stopped: `max_epochs=10` reached.


[NeMo I 2023-05-23 05:09:22 nemo_model_checkpoint:168] New best .nemo model saved to: /home/experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision.nemo


Restoring states from the checkpoint path at /home/experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision--val_wer=0.0053-epoch=8.ckpt
Restored all states from the checkpoint file at /home/experiments/nvidia-stt-ft-02-spe-mixed-precision/2023-05-23_04-57-10/checkpoints/nvidia-stt-ft-02-spe-mixed-precision--val_wer=0.0053-epoch=8.ckpt


CPU times: user 9min 14s, sys: 45.1 s, total: 10min
Wall time: 12min 8s


### Model Evaluation

In [37]:
# Evaluate model on the validation set
trainer.test(asr_model)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[NeMo I 2023-05-21 09:47:12 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:12 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:12 rnnt_wer_bpe:396] predicted :he wrote a book with nine chapters
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:396] predicted :she spent nine days this month volunteering at an animal shelter
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:396] predicted :he had to solve three math problems to pass the test
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:396] predicted :he held up three fingers to indicate how many cupcakes he wanted
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeM

[NeMo W 2023-05-21 09:49:28 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


In [70]:
import numpy as np
import noisereduce as nr
import soundfile as sf
from pyloudnorm import Meter

# Convert manifest files to json format
def manifest_tojson(manifest_file: str) -> str:
    manifest_list = []
    with open(manifest_file, "r") as file:
        for line in file:
            record = json.loads(line)
            manifest_list.append(record)
    return manifest_list

def normalize_to_til(transcript: str) -> str:
    # TIL output is purely uppercase alphabet and space
    # so we normalize the output to that
    result = "".join([c.upper() if c.isalpha() else " " for c in transcript])
    # Remove double spaces
    while "  " in result:
        result = result.replace("  ", " ")
    return result
# def asr_predict(manifest_file, start_idx, end_idx, force_loudness=False, force_denoise=False):
#     predicted_transcripts = []
#     paths = []
#     count = 0
#     for record in manifest_file[start_idx:end_idx]:
#         original_path = record['audio_filepath']
#         path = record['audio_filepath']
#         if force_denoise:
#             data, rate = sf.read(path)
#             data = nr.reduce_noise(audio_clip=data, noise_clip=data, verbose=False)
#             path = path.replace(".wav", "_denoised.wav")
#             sf.write(path, data, rate)
#         transcript, metadata = asr_model.transcribe([path])
#         pred = normalize_to_til(transcript[0])
#         # Attempt to increase loudness if unable to transcribe
#         if pred == "" or force_loudness:
#             count +=1
#             data, rate = sf.read(path)
#             meter = Meter(rate)
#             # measure the loudness
#             loudness = meter.integrated_loudness(data)
#             print("Loudness:", loudness)

#             best_transcription = ""
#             best_target_loudness = None

#             for target_loudness in (-24, -14, -11, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0):
#                 print("Trying target:", target_loudness)
#                 # calculate the gain needed to normalize
#                 gain = target_loudness - loudness
#                 # apply gain correction
#                 normalized_data = data * 10**(gain/20.0)
#                 # write out the normalized audio
#                 if "normalized" not in path:
#                     path = path.replace(".wav", "_normalized.wav")
#                 sf.write(path, normalized_data, rate)
#                 transcript, metadata = asr_model.transcribe([path])
#                 pred = normalize_to_til(transcript[0])
#                 # keep track of the longest transcription and the corresponding loudness
#                 if len(pred) > len(best_transcription) + 4: # if there is a significant change in length (i.e detect new words)
#                     best_transcription = pred
#                     best_target_loudness = target_loudness
#             pred = best_transcription
#             print("Path:", path)
#             print("After normalisation:", best_transcription)
#             print("Best target loudness:", best_target_loudness)
            
#         predicted_transcripts.append(pred)
#         paths.append(original_path[16:]) # ensure original
#     print("Number of audio files too soft:", count)
#     return predicted_transcripts, paths

def asr_predict(manifest_file, start_idx, end_idx, batch_size=4, force_loudness=False, force_denoise=False):
    predicted_transcripts = []
    paths = []
    count = 0
    
    batched_records = [manifest_file[i:i+batch_size] for i in range(start_idx, end_idx, batch_size)]
    
    for batch_records in batched_records:
        batch_paths = []
        for record in batch_records:
            path = record['audio_filepath']
            if force_denoise:
                data, rate = sf.read(path)
                data = nr.reduce_noise(data, rate)
                path = path.replace(".wav", "_denoised.wav")
                sf.write(path, data, rate)
            batch_paths.append(path)

        transcripts, metadata = asr_model.transcribe(batch_paths)

        for original_path, transcript in zip(batch_paths, transcripts):
            pred = normalize_to_til(transcript)
            if pred == "" or force_loudness:
                count +=1
                data, rate = sf.read(original_path)
                meter = Meter(rate)
                loudness = meter.integrated_loudness(data)
                print("Path:", original_path)
                print("Loudness:", loudness)
                
                best_transcription = ""
                best_target_loudness = None
                path = original_path.replace(".wav", "_normalized.wav")

                for target_loudness in (-24, -14, -11, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0):
                    print("Trying target:", target_loudness)
                    gain = target_loudness - loudness
                    normalized_data = data * 10**(gain/20.0)
                    sf.write(path, normalized_data, rate)
                    transcript, metadata = asr_model.transcribe([path])
                    pred = normalize_to_til(transcript[0])
                    if len(pred) > len(best_transcription) + 4:
                        best_transcription = pred
                        best_target_loudness = target_loudness

                print("After normalisation:", best_transcription)
                print("Best target loudness:", best_target_loudness)
                pred = best_transcription
            predicted_transcripts.append(pred)
            paths.append(original_path)
    print("Number of audio files too soft:", count)
    return predicted_transcripts, paths


TEST_MANIFEST = manifest_tojson(test_manifest_file)

In [71]:
import math
BATCH_SIZE = 100
start_idx = 0
end_idx = BATCH_SIZE
iterations = math.ceil(len(TEST_MANIFEST) / BATCH_SIZE)
all_transcripts = []
all_paths = []
asr_model.to("cuda")
for batch in range(iterations):
    print(f'Batch {batch+1}: Records {start_idx} to {end_idx}')
    predicted_transcripts, paths = asr_predict(TEST_MANIFEST, start_idx, end_idx, batch_size=BATCH_SIZE,force_loudness=False, force_denoise=True)
    all_transcripts.extend(predicted_transcripts)
    all_paths.extend(paths)
    start_idx += BATCH_SIZE
    end_idx += BATCH_SIZE

Batch 1: Records 0 to 100
Number of audio files too soft: 0
Batch 2: Records 100 to 200
Number of audio files too soft: 0
Batch 3: Records 200 to 300
Number of audio files too soft: 0
Batch 4: Records 300 to 400
Number of audio files too soft: 0
Batch 5: Records 400 to 500
Number of audio files too soft: 0
Batch 6: Records 500 to 600
Number of audio files too soft: 0
Batch 7: Records 600 to 700
Number of audio files too soft: 0
Batch 8: Records 700 to 800
Number of audio files too soft: 0
Batch 9: Records 800 to 900
Number of audio files too soft: 0
Batch 10: Records 900 to 1000
Number of audio files too soft: 0
Batch 11: Records 1000 to 1100
Number of audio files too soft: 0
Batch 12: Records 1100 to 1200
Number of audio files too soft: 0
Batch 13: Records 1200 to 1300
Number of audio files too soft: 0
Batch 14: Records 1300 to 1400
Number of audio files too soft: 0
Batch 15: Records 1400 to 1500
Number of audio files too soft: 0
Batch 16: Records 1500 to 1600
Number of audio files to

In [ ]:
# Combine all lists of transcripts into a single list to update the annotation column
test_set['annotation'] = list(np.concatenate(all_transcripts).flat)
# Output the submission csv file
test_set.to_csv('eval_submission_nvidia_stt.csv', index=False)

In [73]:
import csv
def create_csv(paths, predictions, output_filename):
    # Ensure that paths and predictions are of the same length
    assert len(paths) == len(predictions), "Lists must have the same length"

    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['path', 'annotation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for path, prediction in zip(paths, predictions):
            writer.writerow({'path': path[16:].replace("_denoised", ""), 'annotation': prediction})
create_csv(all_paths, all_transcripts, 'eval_submission_nvidia_conformer_transducer_ft_mixed_precision_02_normalised_denoise.csv')